Blatt 4
---

Übungstermin: 17. 05. 2017

## Task 4.1 Cramer-Rao Schranke am Boccia-Beispiel:
<font color='#0d2b5b'>

Betrachten Sie das Boccia-Datenmodell aus dem Vorlesungsskript mit $y_i = wx2 + \nu_i$ mit konstanter
Anfangsgeschwindigkeit $x = 0.5$ und dem Schätzer

$$\hat\omega= \frac 1N \sum_{i=1}^N \frac{y_i}{x^2}$$

Dabei sei $\nu_i \sim \mathcal{N} (0, \sigma^2)$ normalverteiltes Rauschen mit Varianz $\sigma^2 = 0.1$.  

</font>

<font color='#0d2b5b'>

Zeige experimentell über je $100$ Wiederholungen, dass für $N \in \{ 1, 2, 3, 5, 10, 100, 1000\}$
die Varianz der Schätzung, d.h. $Var(\hat\omega)$
gegen die Cramer-Rao Schranke konvergiert.
Plotte dazu $Var(\hat\omega)$ und 
$\frac{\sigma^2}{Nx^2}$
in Abhängigkeit von $N$.  

</font>

<font color='#1d478c'>

Wie verändern sich die Schranke und die Schätzungen bei Variation von $\sigma^2$.  

</font>

## Task 4.2 Regularisierte Polynom-Regression:
<font color='#1d478c'>


Berechnen Sie die Regressionsformel für die quadratische Fehlerfunktion

$$
E = \frac12 \sum_{i=1}^N \left( y(x_i) - \hat y(x_i)\right)^2
+\lambda \frac12 \sum_{m=0}^M \hat\omega_m^2 \left(e^m-1\right)
$$
mit speziellem Regularisierungsterm gewichtet mit Parameter $\lambda$ für einen Polynomapproximator


$$
\hat y(x) = \sum_{m_0}^M \hat\omega_m^m
$$
.  

</font>

<font color='#1d478c'>

Welche Idee steckt hinter diesem speziellem Regularisierungsansatz und wie wirkt sich die Regularisierung
auf die Lösungen der Regression aus?

</font>

## Task 4.3 Bias-Varianz Dilemma am Beispiel der Polynom-Regression:
<font color='#1d478c'>

Untersuchen Sie den Einfluss der oben eingeführten Regularisierung auf die Schätzgenauigkeit.  
Als Datenmodell sei das Polynom $y(x, w) + \nu$ gegeben mit wahren Parametern  
$w_0 = 0.8$, $w_1 = 2.4$, $w_2 = −1.55$, $w_3 = −0.15$, $w_4 = 0.1$ und $\nu \sim\mathcal{n} (0, \omega^2 )$
mit $\omega^2 = 2$.  

</font>

<font color='#1d478c'>

Schätzen Sie wiederholt Modellparameter $\hat\omega$ für Polynome 8-ten Grades mittels der regularisierten
Regression aus Aufgabe 3.2. Erstellen Sie dazu $100$ Datensätze bestehend aus je $20$ Datenpunkten
$(x_i , y(x_i ) + \nu_i)$ mit uniform verteilten $x_i \in [−5, 5]$ und wiederholen Sie die Schätzung der Parameter
für jeden Datensatz und Regularisierungen $\lambda \in {0, 10−5 , 10−4 , . . . , 105}$.  

</font>

<font color='#1d478c'>

Welchen Einfluss hat $\lambda$ auf die Aufteilung des Schätzfehlers $h||w − \hat\omega||^2_i$
in Bias- und Varianz-Anteile?
Plotten Sie dazu den Bias des Schätzers, d.h. $||h\hat\omega_i − w||$^2 ,
sowie dessen Varianz $h||\hat\omega − h\hat\omega_i||$^2 i in Abhängigkeit von $\lambda$.  

</font>